In [ ]:
!git clone https://github.com/strivn/idl-project

fatal: destination path 'idl-project' already exists and is not an empty directory.


In [ ]:
# Append paths for the src folder
import sys
import os
sys.path.append("/content/idl-project")

# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'idl-project')))

# Additional imports
from src.utils import *

In [ ]:
!pip install datasets transformers peft accelerate bitsandbytes rouge-score sentencepiece --quiet


In [ ]:
import sys
import os
import torch as t
import numpy as np
import pandas as pd
import torch.nn.functional as F
from tqdm.auto import tqdm

from transformers import GPTNeoXForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer, util

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize

from sentence_transformers import SentenceTransformer
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import re
from scipy.spatial.distance import cosine
from src.model import load_fo_model, load_ba_model, DEVICE
from src.data import load_cnn_dataset
#from src.utils import *
from src.utils_batch_v2 import *
#from src.search import *
from src.search_batch_v2 import *


Using device: cuda


In [ ]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', '{:.4f}'.format)
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
DEVICE

'cuda'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/models/fo_unsup"

fo_model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)
fo_tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

In [ ]:
dataset = load_cnn_dataset(num_samples=95704)
dataset = pd.DataFrame(dataset)
dataset = dataset.iloc[:3000]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset loaded successfully
Example dataset item: {'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places be

In [ ]:
dataset.head(5)

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell...",Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have be...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,"Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a ja...","Mentally ill inmates in Miami are housed on the ""forgotten floor""\nJudge Steven Leifman says most are there as a result of ""avoidable felonies""\nWhile CNN tours facility, patient shouts: ""I am the...",ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who were on the Minneapolis bridge when it collapsed told harrowing tales of survival. ""The whole bridge from one side of the Mississippi to the other just ...","NEW: ""I thought I was going to die,"" driver says .\nMan says pickup truck was folded in half; he just has cut on face .\nDriver: ""I probably had a 30-, 35-foot free fall""\nMinnesota bridge collaps...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,"WASHINGTON (CNN) -- Doctors removed five small polyps from President Bush's colon on Saturday, and ""none appeared worrisome,"" a White House spokesman said. The polyps were removed and sent to the ...","Five small polyps found during procedure; ""none worrisome,"" spokesman says .\nPresident reclaims powers transferred to vice president .\nBush undergoes routine colonoscopy at Camp David .",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,"(CNN) -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appea...","NEW: NFL chief, Atlanta Falcons owner critical of Michael Vick's conduct .\nNFL suspends Falcons quarterback indefinitely without pay .\nVick admits funding dogfighting operation but says he did n...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [ ]:
print(dataset.shape[0])

3000


In [ ]:
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import numpy as np

def linear_attribution_search_fo_only(dataset, fo_model, fo_tokenizer):
    """
    Perform linear attribution search using only the Forward model (Fo).

    For each highlight, find the best matching article sentence using Fo model scoring (reverse direction).
    """
    results = []

    for idx, example in tqdm(dataset.iterrows(), total=len(dataset)):
        # Split article and highlight
        article_sentences = sent_tokenize(example['article'])
        highlight_sentences = sent_tokenize(example['highlights'])

        if not highlight_sentences:
            continue

        highlight = highlight_sentences[0]  # Only use first sentence of highlight

        best_fo_sentence = None
        best_fo_score = float('-inf')

        # Linear search: go through each sentence
        for sentence in article_sentences:
            if len(sentence.split()) < 3:
                continue  # Skip very short sentences

            fo_score = calculate_score(
                highlight, sentence, fo_model, fo_tokenizer,
                backward=False, query_direction="reverse"
            )

            if fo_score['normalized_log_prob'] > best_fo_score:
                best_fo_score = fo_score['normalized_log_prob']
                best_fo_sentence = sentence

        results.append({
            'id': example['id'],
            'highlight': highlight,
            'fo_citation': best_fo_sentence,
            'fo_score': best_fo_score,
            'fo_perplexity': np.exp(-best_fo_score)
        })

    return results

In [ ]:
results_fo_only = linear_attribution_search_fo_only(
    dataset, fo_model, fo_tokenizer
)

  0%|          | 0/3000 [00:00<?, ?it/s]/content/idl-project/src/utils_batch_v2.py:109: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with t.no_grad(), autocast():
100%|██████████| 3000/3000 [2:02:06<00:00,  2.44s/it]


In [ ]:
EMB_MODEL    = SentenceTransformer('all-MiniLM-L6-v2')
ROUGE_SCORER = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
TFIDF_VEC    = TfidfVectorizer(
    token_pattern=r"(?u)\b\w+\b",
    stop_words=None
)

def calculate_embedding_similarity(highlight, citation):
    he = EMB_MODEL.encode([highlight])[0]
    ce = EMB_MODEL.encode([citation])[0]
    return 1 - cosine(he, ce)

def calculate_rouge_score(highlight, citation):
    return ROUGE_SCORER.score(highlight, citation)['rougeL'].fmeasure

def calculate_tfidf_score(highlight, citation):
    try:
        mat = TFIDF_VEC.fit_transform([highlight, citation]).toarray()
        return 1 - cosine(mat[0], mat[1])
    except ValueError:
        return 0.0

def process_data_fo_only(data):
    """
    Process the results when only Fo model is used.
    Calculate embedding, rouge, and tfidf similarities.
    """
    results = []
    for item in data:
        hl = item['highlight']
        rec = {'id': item['id'], 'highlight': hl}

        citation = item.get('fo_citation', None)
        if citation:
            rec['fo_emb_similarity'] = calculate_embedding_similarity(hl, citation)
            rec['fo_rouge_score']     = calculate_rouge_score(hl, citation)
            rec['fo_tfidf_score']     = calculate_tfidf_score(hl, citation)
        else:
            rec['fo_emb_similarity'] = None
            rec['fo_rouge_score']     = None
            rec['fo_tfidf_score']     = None

        results.append(rec)
    return results

linear_final_results = process_data_fo_only(results_fo_only)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import os

df_linear = pd.DataFrame(linear_final_results)

mean_series = df_linear.drop(['id', 'highlight'], axis=1).mean()

data = {
    'Fo_linear': [
        mean_series['fo_emb_similarity'],
        mean_series['fo_rouge_score'],
        mean_series['fo_tfidf_score']
    ]
}

table_df = pd.DataFrame(data, index=['Embedding', 'Rouge', 'Tfidf'])

save_path = "/content/drive/MyDrive/inference"
os.makedirs(save_path, exist_ok=True)

csv_path = os.path.join(save_path, "linear_result.csv")
table_df.to_csv(csv_path, index=True)

print(f"Saved to {csv_path}")

Saved to /content/drive/MyDrive/inference/linear_result.csv


In [ ]:
print(data)

{'Fo_linear': [np.float64(0.6430119872093201), np.float64(0.33885607961749314), np.float64(0.29195307931950604)]}


In [1]:
!git config --global user.name "Shenxuanyi-cmu"
!git config --global user.email "xuanyis@andrew.cmu.edu"

In [2]:
!git clone https://github.com/strivn/idl-project.git
%cd idl-project

Cloning into 'idl-project'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 421 (delta 133), reused 92 (delta 56), pack-reused 215 (from 1)
Receiving objects: 100% (421/421), 15.86 MiB | 26.62 MiB/s, done.
Resolving deltas: 100% (246/246), done.
/content/idl-project


In [5]:
!cp "/content/drive/MyDrive/Colab Notebooks/run_linear_unsup.ipynb" notebooks/

!git add notebooks/run_linear_unsup.ipynb
!git commit -m "Add run_linear_unsup notebook from Colab"

import os
token = getpass.getpass("Enter your GitHub token: ")

repo_url = f"https://{token}@github.com/strivn/idl-project.git"
os.system(f"git remote set-url origin {repo_url}")
os.system("git push origin main")

[main f466e67] Add run_linear_unsup notebook from Colab
 1 file changed, 1 insertion(+)
 create mode 100644 notebooks/run_linear_unsup.ipynb


256